# Divergent data reduction for Amor - advanced

In this notebook, we will look at how to use the `essreflectometry` package with Sciline, for reflectometry data collected from the PSI instrument [Amor](https://www.psi.ch/en/sinq/amor) in [divergent beam mode](https://www.psi.ch/en/sinq/amor/selene).

We will begin by importing the modules that are necessary for this notebook.

## Setup

In [ ]:
# %matplotlib widget
import warnings
import scipp as sc
from ess import amor
from ess.amor import data  # noqa: F401
from ess.reflectometry.types import *
from ess.amor.types import *
from ess.reflectometry import batch_processor

# The files used in this tutorial have some issues that makes scippnexus
# raise warnings when loading them. To avoid noise in the notebook the warnings are silenced.
warnings.filterwarnings('ignore', 'Failed to convert .* into a transformation')
warnings.filterwarnings('ignore', 'Invalid transformation')
warnings.filterwarnings('ignore', 'invalid value encountered')

## Create and configure the workflow

We begin by creating the Amor workflow object which is a skeleton for reducing Amor data,
with pre-configured steps, and then set the missing parameters which are specific to each experiment:

In [ ]:
workflow = amor.AmorWorkflow()
workflow[SampleSize[SampleRun]] = sc.scalar(10.0, unit='mm')
workflow[SampleSize[ReferenceRun]] = sc.scalar(10.0, unit='mm')

workflow[ChopperPhase[ReferenceRun]] = sc.scalar(-7.5, unit='deg')
workflow[ChopperPhase[SampleRun]] = sc.scalar(-7.5, unit='deg')

workflow[WavelengthBins] = sc.geomspace('wavelength', 2.8, 12.5, 2001, unit='angstrom')

# The YIndexLimits and ZIndexLimits define ranges on the detector where
# data is considered to be valid signal.
# They represent the lower and upper boundaries of a range of pixel indices.
workflow[YIndexLimits] = sc.scalar(11), sc.scalar(41)
workflow[ZIndexLimits] = sc.scalar(80), sc.scalar(370)
workflow[BeamDivergenceLimits] = (
    sc.scalar(-0.75, unit='deg'),
    sc.scalar(0.75, unit='deg'),
)

In [ ]:
workflow.visualize(ReflectivityOverQ, graph_attr={'rankdir': 'LR'})

## Caching the reference result

The reference result (used for normalizing the sample data) only needs to be computed once.
It represents the intensity reflected by the super-mirror.

We compute it using the pipeline and thereafter set the result back on the original pipeline.

In [ ]:
workflow[Filename[ReferenceRun]] = amor.data.amor_run(614)
# The sample rotation value in the file is slightly off, so we set it manually
workflow[SampleRotationOffset[ReferenceRun]] = sc.scalar(0.05, unit='deg')

reference_result = workflow.compute(ReducedReference)
# Set the result back onto the pipeline to cache it
workflow[ReducedReference] = reference_result

If we now visualize the pipeline again, we can see that the reference is not re-computed:

In [ ]:
workflow.visualize(ReflectivityOverQ, graph_attr={'rankdir': 'LR'})

## Computing sample reflectivity from batch reduction

We now compute the sample reflectivity from 4 runs that used different sample rotation angles.
The measurements at different rotation angles cover different ranges of $Q$.

We set up a batch reduction helper (using the `batch_processor` function) which makes it easy to process multiple runs at once.

In this tutorial we use some Amor data files we have received.
The file paths to the tutorial files are obtained by calling:

In [ ]:
amor.data.amor_run(608)

When you encounter `amor.data.amor_run` you should imagining replacing that with a path to your own dataset.

In [ ]:
runs = {
    '608': {
        # The sample rotation values in the files are slightly off, so we replace
        # them with corrected values.
        SampleRotationOffset[SampleRun]: sc.scalar(0.05, unit='deg'),
        Filename[SampleRun]: amor.data.amor_run(608),
    },
    '609': {
        SampleRotationOffset[SampleRun]: sc.scalar(0.05, unit='deg'),
        Filename[SampleRun]: amor.data.amor_run(609),
    },
    '610': {
        SampleRotationOffset[SampleRun]: sc.scalar(0.05, unit='deg'),
        Filename[SampleRun]: amor.data.amor_run(610),
    },
    '611': {
        SampleRotationOffset[SampleRun]: sc.scalar(0.05, unit='deg'),
        Filename[SampleRun]: amor.data.amor_run(611),
    },
}

batch = batch_processor(workflow, runs)

# Compute R(Q) for all runs
reflectivity = batch.compute(ReflectivityOverQ)
sc.plot(reflectivity.hist(), norm='log', vmin=1e-5)

In [ ]:
batch.visualize(ReflectivityOverQ, graph_attr={'rankdir': 'LR'})

## Scaling the reflectivity curves to overlap

In case we know the curves are have been scaled by different factors (that are constant in Q) it can be useful to scale them so they overlap:

In [ ]:
# Cache intermediate result to avoid re-loading the data
batch[ReducibleData[SampleRun]] = batch.compute(ReducibleData[SampleRun])

In [ ]:
from ess.reflectometry.tools import scale_for_reflectivity_overlap

scaling = scale_for_reflectivity_overlap(
    reflectivity,
    critical_edge_interval=(
        sc.scalar(0.01, unit='1/angstrom'),
        sc.scalar(0.014, unit='1/angstrom'),
    ),
)

scaled_r = reflectivity * scaling

sc.plot(scaled_r.hist(), norm='log', vmin=1e-5)

Curves obtained from measurements at different angles can be combined to one common reflectivity curve:

In [ ]:
from ess.reflectometry.tools import combine_curves

qbins = sc.geomspace('Q', 0.005, 0.4, 501, unit='1/angstrom')
combined = combine_curves(scaled_r.hist(), qbins)
combined.plot(norm='log')

## Diagnostic figures

There are some useful visualizations that can be used to troubleshoot the instrument.
They typically operate on the `ReflectivityData`.

The difference between `ReflectivityData` and `ReflectivityOverQ` is that `ReflectivityData` is not binned in $Q$, but instead has the same shape as the reference.

Essentially it represents a "reflectivity" computed in every wavelength-detector coordinate (`z_index`) bin.
This makes it easier to spot inhomogeneities and diagnose problems.

In [ ]:
diagnostics = batch.compute(ReflectivityOverZW)
diagnostics['608'].hist().flatten(('blade', 'wire'), to='z').plot(norm='log')

### Make a $(\lambda, \theta)$ map
A good sanity check is to create a two-dimensional map of the counts in $\lambda$ and $\theta$ bins and make sure the triangles converge at the origin.

In [ ]:
from ess.reflectometry.figures import wavelength_theta_figure

wavelength_theta_figure(
    diagnostics.values(),
    theta_bins=batch.compute(ThetaBins[SampleRun]).values(),
    q_edges_to_display=(
        sc.scalar(0.018, unit='1/angstrom'),
        sc.scalar(0.113, unit='1/angstrom'),
    ),
)

This plot can be used to check if the value of the sample rotation angle $\omega$ is correct. The bright triangles should be pointing back to the origin $\lambda = \theta = 0$. In the figure above the black lines are all passing through the origin.

### Make a $(Q, \theta)$ map
Another good sanity check is to create a two-dimensional map of the counts in $\lambda$ and $Q$ and make sure the stripes are vertical. If they are not that could indicate that the `ChopperPhase` setting is incorrect.

In [ ]:
from ess.reflectometry.figures import q_theta_figure

q_theta_figure(
    diagnostics.values(),
    theta_bins=batch.compute(ThetaBins[SampleRun]).values(),
    q_bins=qbins,
)

### Compare the sample measurement to the reference on the detector

Here we compare the raw number of counts on the detector for the sample measurement and the reference respectively.

`z_index` is the $z-$detector coordinate, that is, the detector coordinate in the direction of the scattering angle $\theta$.

In [ ]:
from ess.reflectometry.figures import wavelength_z_figure

wf_608 = batch.workflows['608']

wavelength_z_figure(
    wf_608.compute(Sample), wavelength_bins=wf_608.compute(WavelengthBins), grid=False
) + wavelength_z_figure(wf_608.compute(Reference), grid=False)

## Save data

We can save the computed $I(Q)$ to an [ORSO](https://www.reflectometry.org) [.ort](https://github.com/reflectivity/file_format/blob/master/specification.md) file using the [orsopy](https://orsopy.readthedocs.io/en/latest/index.html) package.

First, we need to collect the metadata for that file.
To this end, we insert a parameter to indicate the creator of the processed data.

In [ ]:
from ess.reflectometry import orso
from orsopy import fileio

In [ ]:
batch[orso.OrsoCreator] = {
    k: orso.OrsoCreator(
        fileio.base.Person(
            name='Max Mustermann',
            affiliation='European Spallation Source ERIC',
            contact='max.mustermann@ess.eu',
        )
    )
    for k in runs
}

We can visualize the workflow for a single run (`'608'`):

In [ ]:
wf_608.visualize(orso.OrsoIofQDataset, graph_attr={'rankdir': 'LR'})

In [ ]:
# We want to save the scaled results to the file
batch[ReflectivityOverQ] = scaled_r

We build our ORSO dataset from the computed $I(Q)$ and the ORSO metadata:

In [ ]:
iofq_datasets = batch.compute(orso.OrsoIofQDataset)

We also add the URL of this notebook to make it easier to reproduce the data:

In [ ]:
for ds in iofq_datasets.values():
    ds.info.reduction.script = (
        'https://scipp.github.io/essreflectometry/user-guide/amor/amor-reduction.html'
    )

Finally, we can save the data to a file.
Note that `iofq_datasets` contains [orsopy.fileio.orso.OrsoDataset](https://orsopy.readthedocs.io/en/latest/orsopy.fileio.orso.html#orsopy.fileio.orso.OrsoDataset)s.

In [ ]:
fileio.orso.save_orso(
    datasets=list(iofq_datasets.values()), fname='amor_reduced_iofq.ort'
)

Look at the first 50 lines of the file to inspect the metadata:

In [ ]:
!head amor_reduced_iofq.ort -n50

## Merging multiple runs for the same sample rotation

In [ ]:
runs = {
    '608': {
        # The sample rotation values in the files are slightly off, so we replace
        # them with corrected values.
        SampleRotationOffset[SampleRun]: sc.scalar(0.05, unit='deg'),
        Filename[SampleRun]: amor.data.amor_run(608),
    },
    '609': {
        SampleRotationOffset[SampleRun]: sc.scalar(0.05, unit='deg'),
        Filename[SampleRun]: amor.data.amor_run(609),
    },
    '610': {
        SampleRotationOffset[SampleRun]: sc.scalar(0.05, unit='deg'),
        Filename[SampleRun]: amor.data.amor_run(610),
    },
    '611+612': {
        SampleRotationOffset[SampleRun]: sc.scalar(0.05, unit='deg'),
        Filename[SampleRun]: (
            amor.data.amor_run(611),
            amor.data.amor_run(612),
        ),  # List of files means their event lists should be merged
    },
}

batch = batch_processor(workflow, runs)

# Compute R(Q) for all runs
reflectivity = batch.compute(ReflectivityOverQ)
sc.plot(reflectivity.hist(), norm='log', vmin=1e-4)

In [ ]:
batch.visualize(ReflectivityOverQ, graph_attr={'rankdir': 'LR'})

In [ ]:
scaled_r = reflectivity * scale_for_reflectivity_overlap(
    reflectivity,
    critical_edge_interval=(
        sc.scalar(0.01, unit='1/angstrom'),
        sc.scalar(0.014, unit='1/angstrom'),
    ),
)

sc.plot(scaled_r.hist(), norm='log', vmin=1e-5)